# Import Library

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

DS to LD

In [2]:
cwd = os.getcwd()
# 폴더 위치 변경
os.chdir("C:\\Users\hyunj\\Dropbox\\종합설계프로젝트\\중요\\데이터")

data = pd.read_csv('DStoLD.csv', encoding='cp949')
print('LD의 Feature : ' , data.columns)
data.head()

LD의 Feature :  Index(['Case ID', 'Activity', 'Start Timestamp', 'Complete Timestamp',
       'Variant', 'Variant index', 'VESSEL', 'MCHN_ID', 'FULL_EMPTY', 'POD',
       'BLOCK_BAY'],
      dtype='object')


,Case ID,Activity,Start Timestamp,Complete Timestamp,Variant,Variant index,VESSEL,MCHN_ID,FULL_EMPTY,POD,BLOCK_BAY
0,BEAU2976719-1,DS-QUAYSIDE,2018-01-27 16:00:39.000,2018-01-27 16:02:11.000,Variant 1,1,MAUO-001/2018,GC101,F,KRPUS,2B-19
1,BEAU2976719-1,DS-MOVE,2018-01-27 16:02:11.000,2018-01-27 16:10:10.000,Variant 1,1,MAUO-001/2018,YT586,F,KRPUS,2B-19
2,BEAU2976719-1,DS-YARDSIDE,2018-01-27 16:10:10.000,2018-01-27 16:11:42.000,Variant 1,1,MAUO-001/2018,TC223,F,KRPUS,2B-19
3,BEAU2976719-1,LD-YARDSIDE,2018-01-31 20:01:47.000,2018-01-31 20:04:20.000,Variant 1,1,ALAP-001/2018,TC223,F,CAVAN,2B-19
4,BEAU2976719-1,LD-MOVE,2018-01-31 20:04:20.000,2018-01-31 20:21:20.000,Variant 1,1,ALAP-001/2018,YT511,F,CAVAN,2B-19


In [3]:
# 필요없는 열 삭제 & Feature name 변경
data.drop('Variant', axis = 1, inplace = True)
data.drop('Variant index', axis = 1, inplace = True)

data.rename(columns = {'Case ID':'CONTAINER_NO',
                       'Activity':'JOB_TYPE',
                       'Start Timestamp': 'JOB_START_DT',
                       'Complete Timestamp' : 'JOB_END_DT'},
                        inplace=True)
print('DStoLD의 Feature : ' , data.columns)
print('DStoLD의 SHAPE : ', data.shape)
data.head()

DStoLD의 Feature :  Index(['CONTAINER_NO', 'JOB_TYPE', 'JOB_START_DT', 'JOB_END_DT', 'VESSEL',
       'MCHN_ID', 'FULL_EMPTY', 'POD', 'BLOCK_BAY'],
      dtype='object')
DStoLD의 SHAPE :  (149400, 9)


,CONTAINER_NO,JOB_TYPE,JOB_START_DT,JOB_END_DT,VESSEL,MCHN_ID,FULL_EMPTY,POD,BLOCK_BAY
0,BEAU2976719-1,DS-QUAYSIDE,2018-01-27 16:00:39.000,2018-01-27 16:02:11.000,MAUO-001/2018,GC101,F,KRPUS,2B-19
1,BEAU2976719-1,DS-MOVE,2018-01-27 16:02:11.000,2018-01-27 16:10:10.000,MAUO-001/2018,YT586,F,KRPUS,2B-19
2,BEAU2976719-1,DS-YARDSIDE,2018-01-27 16:10:10.000,2018-01-27 16:11:42.000,MAUO-001/2018,TC223,F,KRPUS,2B-19
3,BEAU2976719-1,LD-YARDSIDE,2018-01-31 20:01:47.000,2018-01-31 20:04:20.000,ALAP-001/2018,TC223,F,CAVAN,2B-19
4,BEAU2976719-1,LD-MOVE,2018-01-31 20:04:20.000,2018-01-31 20:21:20.000,ALAP-001/2018,YT511,F,CAVAN,2B-19


In [4]:
# 통합할 데이터 호출
che_data = pd.read_csv('CHE I perspective.csv')
gate_data = pd.read_csv('Gate perspective.csv')
qc_data = pd.read_csv('QC perspective.csv')
vessel_data = pd.read_csv('Vessel perspective.csv')

In [5]:
# 최종 DS csv 파일 - 자세한 것은 파워포인트 참고
temp_df = pd.DataFrame(columns=["CONTAINER_NO", "VESSEL", "MCHN_ID", "FULL_EMPTY", "JOB_TYPE", "POD", "BLOCK_BAY", "JOB_START_DT", "JOB_END_DT",
                                "CHE_ID", "ITV_ID", "QC_ID", "OUTSERVICE", "QUEUE_POSITION", "H_BOX",
                                "OUTVESSEL", "COUNTRY_NEXTPOD", "PORT_NEXTPOD", "TML_OUT_DT"])
print('최종적으로 사용할 데이터 셋의 Column 수는' , len(temp_df.columns))

# temp에서 추가하고 이를 last에서 concat하는 걸로 - 왜냐하면 temp에서 containerid별로 가져올 것이기 때문에, 이렇게 해주는 이유는 정렬하기 위해서 
# (사실 다 추가하고 정렬하면 괜찮지만, 작업 중간 중간에 볼 것이기 때문에 이렇게 함 - 메모리 이슈는 고려하지 않음) 
last_df = temp_df.copy()

최종적으로 사용할 데이터 셋의 Column 수는 19


In [6]:
unique_container_list = list(data['CONTAINER_NO'].unique())
print(len(unique_container_list))

24900


In [7]:
csv_list = [gate_data, che_data, qc_data]

for csv in csv_list :
    temp = max(csv['CONTAINER_NO'].value_counts())
    print(temp)

2
3
2


In [8]:
only_gate_data = gate_data[gate_data['CONTAINER_NO'].isin(unique_container_list)]
print(len(only_gate_data['CONTAINER_NO'].unique()))
value_counts_gate = only_gate_data['CONTAINER_NO'].value_counts()

4


In [9]:
only_che_data = che_data[che_data['CONTAINER_NO'].isin(unique_container_list)]
print(len(only_che_data['CONTAINER_NO'].unique()))
value_counts_che = only_che_data['CONTAINER_NO'].value_counts()

24900


In [10]:
# CHE에 대한 정보가 있는 것들
print(len(value_counts_che[value_counts_che==2]))
print(len(value_counts_che[value_counts_che==3]))

1379
23521


In [11]:
only_qc_data = qc_data[qc_data['CONTAINER_NO'].isin(unique_container_list)]
print(len(only_qc_data['CONTAINER_NO'].unique()))
value_counts_qc = only_qc_data['CONTAINER_NO'].value_counts()

24900


In [12]:
only_vessel_data = vessel_data[vessel_data['CONTAINER_NO'].isin(unique_container_list)]
print(len(only_vessel_data['CONTAINER_NO'].unique()))
value_counts_vessel = only_vessel_data['CONTAINER_NO'].value_counts()

21181


CHE에서 연관되는 행이 3개 - 하지만 이 중에서 GO & GI에 대한 정보는 없음 <br>
gate perspective에서 매칭되는 컨테이너 아이디가 4개뿐임 - 일단 연결하는 것으로 하엿

In [13]:
# 데이터 우선 통합 - Container perspective 기준!
temp_df = pd.concat([temp_df,data])

In [14]:
# perspetive를 비교하면서 매치되지 않는 CONTAINER_NO 개수를 기록하기 위한 변수 선
for container in tqdm(unique_container_list, desc = "데이터 통합 중") : 
    criteria_df = temp_df[temp_df['CONTAINER_NO'] == container].reset_index(drop=True)
    #! Gate perspective에서 정보 추출 - GO & GI (DS에선 GO!)
    gate_temp_df = gate_data[gate_data['CONTAINER_NO'] == container].reset_index(drop=True)
    
    # container perspective에 있는 CONTAINER_NO가 gate_perspective에 없는 경우도 고려해보아야 함
    # 이 경우에는 4개만 추가
    if len(gate_temp_df) != 0 :
        criteria_df = pd.concat([criteria_df, gate_temp_df])
        criteria_df = criteria_df.reset_index(drop=True)
    
    #! CHE I perspective에서 정보 추출 - MOVE
    #* DStoLD의 CHE I perspective에는 LD의 MOVE과 DS의 MOVE의 정보가 있음
    che_temp_df = che_data[che_data['CONTAINER_NO'] == container].reset_index(drop=True)
    
    ld_move_che_temp_df = che_temp_df[che_temp_df['JOB_TYPE'] == "LD"].reset_index(drop=True)
    ds_move_che_temp_df = che_temp_df[che_temp_df['JOB_TYPE'] == "DS"].reset_index(drop=True)
    # MOVE와 연관이 있는 것을 확인하였지만, 혹시 모르니 두 개의 ID를 비교하여 정보 추가
    # 시간으로 하니 정확한 비교가 안됨
    if(len(che_temp_df) != 0) :
        # LD move에 관한 정보가 있으면
        if (len(ld_move_che_temp_df) != 0) :
            che_id = ld_move_che_temp_df.loc[0, 'MCHN_ID']
            itv_id = ld_move_che_temp_df.loc[0, 'ITV_ID']
            qc_id = ld_move_che_temp_df.loc[0, 'QC_ID']
            criteria_df.loc[4, 'CHE_ID'] = che_id
            criteria_df.loc[4, 'ITV_ID'] = itv_id
            criteria_df.loc[4, 'QC_ID'] = qc_id
                
        # DS move에 관한 정보가 있으면
        if (len(ds_move_che_temp_df) != 0) :
            che_id = ds_move_che_temp_df.loc[0, 'MCHN_ID']
            itv_id = ds_move_che_temp_df.loc[0, 'ITV_ID']
            qc_id = ld_move_che_temp_df.loc[0, 'QC_ID']
            criteria_df.loc[1, 'CHE_ID'] = che_id
            criteria_df.loc[1, 'ITV_ID'] = itv_id
            criteria_df.loc[1, 'QC_ID'] = qc_id
    
    #! QC perspective에서 정보 추출 - QUAYSIDE
    qc_temp_df = qc_data[qc_data['CONTAINER_NO'] == container].reset_index(drop=True)
    
    ld_qc_temp_df = qc_temp_df[qc_temp_df['JOB_TYPE'] == "L"].reset_index(drop=True)
    ds_qc_che_temp_df = qc_temp_df[qc_temp_df['JOB_TYPE'] == "D"].reset_index(drop=True)
    
    # 2번의 QUAYSIDE가 있음    
    ld_queue_position = ld_qc_temp_df.loc[0, 'QUEUE_POSITION']
    ld_h_box = ld_qc_temp_df.loc[0, 'H_BOX']
    criteria_df.loc[5, 'QUEUE_POSITION'] = ld_queue_position
    criteria_df.loc[5, 'H_BOX'] = ld_h_box
    
    ds_queue_position = ds_qc_che_temp_df.loc[0, 'QUEUE_POSITION']
    ds_h_box = ds_qc_che_temp_df.loc[0, 'H_BOX']
    criteria_df.loc[0, 'QUEUE_POSITION'] = ds_queue_position
    criteria_df.loc[0, 'H_BOX'] = ds_h_box
    
    #! Vessel perspective - 여전히 DS에 관한 것만 있음
    vessel_temp_df = vessel_data[vessel_data['CONTAINER_NO'] == container].reset_index(drop=True)
    
    if(len(vessel_temp_df) != 0) :
        out_vessel = vessel_temp_df.loc[0, 'OUTVESSEL']
        country_nextpod = vessel_temp_df.loc[0, 'COUNTRY_NEXTPOD']
        port_nextpod = vessel_temp_df.loc[0, 'PORT_NEXTPOD']
        tml_out_dt = vessel_temp_df.loc[0, 'TML_OUT_DT']
        
        criteria_df.loc[0, 'OUTVESSEL'] = out_vessel
        criteria_df.loc[0, 'COUNTRY_NEXTPOD'] = country_nextpod
        criteria_df.loc[0, 'PORT_NEXTPOD'] = port_nextpod
        criteria_df.loc[0, 'TML_OUT_DT'] = tml_out_dt
    
    
    #! 각 정보를 전달한 뒤에 last_df에 concat
    last_df = pd.concat([criteria_df, last_df])
    last_df = last_df.reset_index(drop=True)
    

데이터 통합 중: 100%|██████████| 24900/24900 [29:52<00:00, 13.89it/s]


In [15]:
last_df.to_csv('Integrated DStoLD.csv', encoding='cp949', index=False)